In [1]:
using CSV
using Distances
using Random

In [2]:
Random.seed!(1)
n = 100      # The number of patients
c_2 = 5     # unit distance cost
patients = [2:n+1;]
V = [1:n+1;]  # 1 denotes depot.    
q = [rand(12:15,n+1);]   # Requirment of each patient
arcs = []
for i in V
    for j in V 
        push!(arcs,(i,j))
    end
end
Q = 100    # Capacity
D = 10  # Max flying distance
q

101-element Array{Int64,1}:
 14
 14
 15
 13
 13
 14
 14
 15
 14
 15
 12
 12
 12
  ⋮
 15
 13
 12
 14
 14
 12
 14
 12
 12
 14
 13
 12

In [3]:
data = CSV.read("larger_numbers_patients_21239_TableToExcel.csv")

,OID,CID,POINT_X,POINT_Y
,Int64,Int64,Float64,Float64
1,0,1,-76.5831,39.3669
2,1,1,-76.5937,39.3583
3,2,1,-76.5833,39.3641
4,3,1,-76.5897,39.3493
5,4,1,-76.5819,39.374
6,5,1,-76.5941,39.3812
7,6,1,-76.5799,39.3808
8,7,1,-76.5853,39.3614
9,8,1,-76.5741,39.3649


In [4]:
x_loc = data[:,3];
y_loc = data[:,4];
d = zeros(n+1,n+1)   # Distance matrix
for i = 1: length(V)
    for j = 1: length(V)
        d[i,j] = haversine((x_loc[i],y_loc[i]),(x_loc[j],y_loc[j]),6372.8) 
    end
end
d

101×101 Array{Float64,2}:
 0.0       1.32421   0.312517  2.042     …  1.12805   0.929697  0.881999
 1.32421   0.0       1.1054    1.0594       1.0015    0.917244  1.27315 
 0.312517  1.1054    0.0       1.73906      0.818631  0.95109   1.02077 
 2.042     1.0594    1.73906   0.0          1.05465   1.95252   2.28653 
 0.797621  2.02274   1.10954   2.83267      1.917     1.32445   1.0432  
 1.8542    2.54846   2.12022   3.57108   …  2.90346   1.63122   1.2846  
 1.5665    2.76665   1.87724   3.60296      2.6762    1.98543   1.64637 
 0.642309  0.801809  0.347572  1.39983      0.575894  0.934604  1.12231 
 0.798762  1.83626   0.790075  2.19523      1.14058   1.70594   1.67937 
 1.22463   1.52913   0.977858  1.48596      0.542155  1.84747   1.98712 
 1.30496   2.51691   1.61565   3.3423    …  2.41555   1.75938   1.43346 
 0.669373  1.78738   0.70627   2.22147      1.17172   1.59039   1.5442  
 0.773525  1.04127   0.845725  2.04333      1.46574   0.195509  0.245011
 ⋮                       

In [ ]:
using JuMP, Gurobi 
v = 1:15  # # of drones
m = Model(Gurobi.Optimizer);
@variable(m,x[arcs,v],Bin) #arcs covered by vehicle k
@variable(m,u[1:n+1,v])
# Each patient will be visited exactly once
@constraint(m,single1[j in patients],sum(x[(i,j),k] for i in V for k in v)==1) 
@constraint(m,single2[i in patients],sum(x[(i,j),k] for j in V for k in v)==1) 
# Drone starts from depot
@constraint(m,depot1[k in v],sum(x[(1,j),k] for j in patients)==1)
# Drone ends at depot
@constraint(m,depot2[k in v],sum(x[(i,1),k] for i in patients)==1)
# Capacity constriant
@constraint(m,capacity[k in v],sum(q[i]*x[(i,j),k] for i in patients for j in V)<=Q)
# Balance constraint
@constraint(m,balance[h in patients,k in v],sum(x[(i,h),k] for i in V)-sum(x[(h,j),k] for j in V)==0) 
# Subtour Elimination MTZ
@constraint(m,subtour1[k in v], u[1,k]==1)
@constraint(m,subtour2[k in v,i in 2:n+1], u[i,k]>=2)   
@constraint(m,subtour3[k in v,i in 2:n+1], u[i,k]<=n)
@constraint(m,subtour4[k in v,i in 2:n+1, j in 2:n+1], (u[i,k]-u[j,k]+1)<=(n-1)*(1-x[(i,j),k]))
# Maximum flying distance for each drone
@constraint(m,maxdis[k in v], sum(d[i,j]*x[(i,j),k] for i in V for j in V)<=D)                                             
# Objective Function
@objective(m,Min,5*sum(d[i,j]*x[(i,j),k] for i in V for j in V for k in v))                                                              
optimize!(m)
@show objective_value(m);

In [ ]:
val = value.(x)
for k in v
    for i in V
        for j in V
            if val[(i,j),k]==1
                println(x[(i,j),k])
            end 
        end
    end
end